In [17]:
# Useful starting lines
%matplotlib inline
import random
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce
from svm_function import contains_CCAT, calculate_loss, is_support, gradient_update, mini_batch_update, calculate_accuracy
import data
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### Vector initialization (using dict formats)
Generate batch

In [5]:
batch_size = 1000
iterator = data.get_batch(batch_size)

# Pour avoir le prochain batch of size "batch_size"
batch = next(iterator)

# Pour avoir un sample
# Not used anymore
one_batch = dict(list(batch.values())[51])

Generate labels

In [6]:
labels = data.load_labels()
final_labels = dict(map(contains_CCAT,labels.items()))

Random initialization for weight vector

In [7]:
num_features = 47236
w = np.random.uniform(low=-1, high = 1, size = num_features)
dict_w={}
for (i,j) in enumerate(w):
    dict_w[i+1]=j

#### Testing functions
Loss computation

In [8]:
calculate_loss(final_labels, batch, dict_w)

972.68609889872664

Mini batch update

In [13]:
mini_batch_update(batch, final_labels, dict_w, 50)

{40961: 0.221487766318644,
 12291: 0.0678936589463246,
 12293: -0.0714771127114971,
 12294: -0.0421070806950073,
 16395: -0.0382478436765617,
 14338: 0.0361867462580607,
 4111: -0.108297187841695,
 12307: 0.178858214157608,
 4125: -0.0384436349682508,
 12318: -0.0953128599798187,
 28703: -0.0295283582788246,
 27312: -0.092656346740054,
 4130: -0.0873419441840431,
 27995: -0.0868914686595095,
 10929: 0.0788325092852693,
 4139: -0.146738439956642,
 32815: -0.216588025973681,
 32816: -0.0504115404414686,
 12337: 0.0860129483201019,
 20531: -0.0778996819821216,
 32820: 0.25338535792602,
 41013: -0.0937007882576678,
 9567: 0.100302411904367,
 12352: -0.046256059879732,
 12354: -0.0506637502122012,
 28740: -0.0816313149715121,
 12358: -0.0766105479046506,
 8265: -0.0880858029428428,
 28747: 0.034889229396216,
 24652: 0.207794278956355,
 28750: -0.158993151149924,
 28753: 0.0298590895836458,
 8274: 0.152473554879562,
 83: -0.104367021842678,
 36948: -0.195683157164153,
 20568: 0.0513432261967

Gradient update (before)

In [176]:
#is_support(-1, one_batch, dict_w)
#gradient_update(-1, one_batch, dict_w)

### With numpy (before)

In [ ]:
# hinge loss function, but we could also use more standards loss later 
def hinge_loss(y, X, w):
    return np.clip(1 - y * (X @ w), 0, np.inf)

def calculate_primal_objective(y, X, w, lambda_):
    """compute the full cost (the primal objective), that is loss plus regularizer.
    X: the full dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    """
    v = hinge_loss(y, X, w)
    return np.sum(v) + lambda_ / 2 * np.sum(w ** 2)
    
def accuracy(y1, y2):
    return np.mean(y1 == y2)

def prediction(X, w):
    return (X @ w > 0) * 2 - 1

def calculate_accuracy(y, X, w):
    predicted_y = prediction(X, w)
    return accuracy(predicted_y, y)

In [36]:
def sgd_for_svm_demo(y, X):
    
    max_iter = 500000
    gamma = 1
    lambda_ = 0.00001
    random.seed(2)
    
    num_examples, num_features = X.shape
    w = np.zeros(num_features)
    
    # convert to list for main loop
    w = list(w)
    
    def is_support(y_n, x_n, w):
        """a datapoint is support if max{} is not 0. """
        dot_prod = 0
        for (x_n_i, w_i) in zip(x_n,w):
            dot_prod += x_n_i*w_i
        return y_n * dot_prod < 1
    
    for it in range(max_iter):
        
        # n = sample one data point uniformly at random data from x
        n = random.randint(0,num_examples-1)
        
        x_n, y_n = X[n], y[n]
        
        # convert to list for main loop
        x_n = list(x_n)
        
        ################ MAIN LOOP ##################
        ############# GRADIENT UPDATE ###############
        
        if(is_support(y_n, x_n, w)):
            grad = [- y_n * x_n_i + lambda_ * w_i for (x_n_i ,w_i) in zip(x_n,w)]
            w = [w_i - gamma/(it+1) * grad_i for (w_i,grad_i) in zip(w,grad)]
        
        else:
            grad = [lambda_ * w_i for w_i in w]
            w = [w_i - gamma/(it+1) * grad_i for (w_i,grad_i) in zip(w,grad)]
        ################ END OF LOOP ################
        
        if it % 10000 == 0:
            cost = calculate_primal_objective(y, X, np.array(w), lambda_)
            print("iteration={i}, cost={c}".format(i=it, c=cost))
    
    print("training accuracy = {l}".format(l=calculate_accuracy(y, X, w)))

sgd_for_svm_demo(y, X)

iteration=0, cost=8379144195.244217
iteration=10000, cost=2569445.6572731775
iteration=20000, cost=1606766.2442051857
iteration=30000, cost=1186094.9340004118
iteration=40000, cost=1007087.3735559117
iteration=50000, cost=1131750.4592788236
iteration=60000, cost=746568.3302071713
iteration=70000, cost=686601.2487545054
iteration=80000, cost=639245.6090707903
iteration=90000, cost=609544.1412513953
iteration=100000, cost=592931.1118818587
iteration=110000, cost=573526.3415829436
iteration=120000, cost=544963.1939392536
iteration=130000, cost=561023.8797291429
iteration=140000, cost=540350.6845321481
iteration=150000, cost=507792.3903086364
iteration=160000, cost=502465.41475056723
iteration=170000, cost=517028.95424482174
iteration=180000, cost=493117.70732193266
iteration=190000, cost=480501.13065999973
iteration=200000, cost=474716.177453672
iteration=210000, cost=463009.580368568
iteration=220000, cost=489125.67552255426
iteration=230000, cost=453521.6787539215
iteration=240000, cost

## Stochastic Gradient Descent for SVM
#### Version du lab du cours de ML - inspiration

In [ ]:
def calculate_stochastic_gradient(y, X, w, lambda_, n):
    """compute the stochastic gradient of loss plus regularizer.
    X: the dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    n: the index of the (one) datapoint we have sampled
    """
    # Be careful about the constant N (size) term!
    # The complete objective for SVM is a sum, not an average as in earlier SGD examples!
    def is_support(y_n, x_n, w):
        """a datapoint is support if max{} is not 0. """
        return y_n * x_n @ w < 1
    
    x_n, y_n = X[n], y[n]
    
    grad = - y_n * x_n.T if is_support(y_n, x_n, w) else np.zeros_like(x_n.T)
    grad = np.squeeze(grad) + lambda_ * w
    
    return grad